## Flow for paper modelling different PPA scenarios

Outline:
1. Basic data analysis: quality of data, basic statistics. For both load and gen
2. Select the scenario to be modelled. Variables to define include:
    * PPA delivery structure (PaP, shaped, baseload)
    * Use of (and params around) demand shifting
    * Whether a battery is added and what size
3. Create the 'optimal' hybrid profile according to the delivery structure:
    * For PaP - 'optimal' is best match to customer load
    * For shaped - 'optimal' is best match to customer load (maybe? Or is this just most consistent shape, i.e. most reliable to deliver? Probably this but idk how this would work)
    * For baseload - 'optimal' is best match to contract shape
4. If load shifting and/or battery operation is involved: add that here
5. Compare traces to find key values:
    * Matched and unmatched load
    * Matched and unmatched contracted generation
    * Wholesale value of each match/unmatch load and gen
    * Emissions associated with unmatched load
6. Calculate reasonable strike price(s) including risk-based premiums. Decide and define firming contract
7. Run ppa.calc scripts including firming contract details to receive financial outcomes
8. See what happens!

In [1]:
# ------------------------------ Packages & Files ------------------------------
import pandas as pd
import numpy as np
import seaborn as sns
import seaborn.objects as so
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import nemed
import ppa, residuals, tariffs, hybrid, firming_contracts
import calendar
import holidays
import pprint
from mip import Model, xsum, minimize, CONTINUOUS, BINARY, conflict
from nemosis import dynamic_data_compiler, static_table
from typing import List
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from getting_data import import_gen_data, import_load_data, import_pricing_data, import_emissions_data
from helper_functions import _check_interval_consistency, _check_missing_data, get_interval_length

pd.set_option('display.max_rows', None)

INFO: Using Python-MIP package version 1.15.0


In [2]:
# -------------------------------- USER INPUTS ---------------------------------

# - - - - - - - - - - - - - - - - - - DATA - - - - - - - - - - - - - - - - - - -

# load data file name
LOAD_FN = '/Users/elliekallmier/Desktop/RA_Work/247_TRUZERO/247_ppa/five_year_data/Engineering-Metal and non-metal fabrications_86_SW_same_year.csv'
LOAD_COL_NAME = 'Load'
LOAD_TIMEZONE = ''

# generation data file name, or, generator DUID(s)
# TODO: add function to take input TYPE of generator and select appropriate trace
GEN_FN = ''
GEN_COL_NAME_S = ''

# get name of datetime column for both load and generation files
LOAD_DATETIME_COL_NAME = 'TS'
GEN_DATETIME_COL_NAME = ''
DAY_FIRST = True
GEN_TECH_TYPE_S = ['WIND - ONSHORE', 'PHOTOVOLTAIC FLAT PANEL']

# Region to fetch generation/pricing/emissions data for:
REGION = 'QLD1'

# NEMOSIS inputs:
RAW_DATA_CACHE = 'data_caches/gen_data_cache'
EMISSIONS_CACHE = 'data_caches/nemed_cache'
PRICING_CACHE = 'data_caches/pricing_cache'


# - - - - - - - - - - - - - - - - - CONTRACT - - - - - - - - - - - - - - - - - -
# TODO: fill out
DELIVERY_STRUCTURE = ''
FIRMING_CONTRACT_TYPE = 'Partially wholesale exposed'

EXPOSURE_BOUND_UPPER = 300
EXPOSURE_BOUND_LOWER = 20
RETAIL_TARIFF_DETAILS = {'a':'b'}

FLOOR_PRICE = 0.0



In [ ]:
# -------------------------------- Get Load Data -------------------------------
#   - check dtypes of columns - should all be float, except datetime col.
#   - update colname(s)
#   - set datetime index
#   - get interval length
#   - check for NaN/missing data

load_data, interval, start_date, end_date = import_load_data.get_load_data(LOAD_FN, LOAD_DATETIME_COL_NAME, LOAD_COL_NAME, DAY_FIRST)

In [ ]:
# ----------------------------- Get Generation Data ----------------------------
gen_data = import_gen_data.get_generation_data(RAW_DATA_CACHE, REGION, GEN_TECH_TYPE_S, interval, start_date=start_date, end_date=end_date)

In [ ]:
# ------------------------ Get Pricing & Emissions Data ------------------------
emissions_intensity = import_emissions_data.get_avg_emissions_intensity(
    start_date, end_date, EMISSIONS_CACHE, [REGION], period=f'{interval}min'
)

In [6]:
price_data = import_pricing_data.get_wholesale_price_data(
    start_date, end_date, PRICING_CACHE, [REGION], period=f'{interval}min'
)

INFO: Compiling data for table DISPATCHPRICE
INFO: Returning DISPATCHPRICE.


In [176]:
load_data = load_data.resample('H').sum(numeric_only=True)
gen_data = gen_data.resample('H').sum(numeric_only=True)
emissions_intensity = emissions_intensity.resample('H').mean(numeric_only=True)
price_data = price_data.resample('H').mean(numeric_only=True)

In [177]:
# Now combine all of the data together:
combined_data = pd.concat([load_data, gen_data, price_data, emissions_intensity], axis='columns')

In [178]:
combined_data_firming = firming_contracts.choose_firming_type(
    FIRMING_CONTRACT_TYPE, combined_data, [REGION], EXPOSURE_BOUND_UPPER, EXPOSURE_BOUND_LOWER, RETAIL_TARIFF_DETAILS
)

In [179]:
combined_data_firming.columns

Index(['Load', 'CSPVPS1: Photovoltaic Flat Panel',
       'DDSF1: Photovoltaic Flat Panel', 'KSP1: Photovoltaic Flat panel',
       'MEWF1: Wind - Onshore', 'SMCSF1: Photovoltaic Flat panel',
       'SRSF1: Photovoltaic Flat panel', 'RRP: QLD1', 'AEI: QLD1',
       'Firming price: QLD1'],
      dtype='object')

In [9]:
# ------------------------- CONTRACT DELIVERY STRUCTURE ------------------------

# choice between: PaP, Shaped, Baseload, 24/7, PaC
# Shaped and Baseload can both be re-scaled annually/quarterly/monthly as desired

In [180]:
def check_leap_year(
        df:pd.DataFrame,
        intervals_in_day:int
) -> bool:
    day_one = df.index[0]
    day_365 = day_one + timedelta(days=365)

    return ~(day_one.day == day_365.day)

In [241]:
def run_hybrid_optimisation(
        contracted_energy:pd.Series,
        wholesale_prices:pd.Series,
        generation_data:pd.DataFrame,
        excess_penalty:float,
        total_sum:float,
        contract_type:str,
        constrain_total_percent:bool=False,
        cfe_score_min:float=None
        # constrain_total_sum:bool=True
) -> tuple[pd.Series, dict[str:dict[str:float]]]:
    # TODO: consider if this return structure is actually best/fit for purpose here
    gen_names = {}
    gen_data_series = {}
    wholesale_prices = np.array(wholesale_prices.clip(lower=0.0).values)

    market_cap = 16600  # market price cap value to use as oversupply penalty

    for _, gen in enumerate(generation_data):
        gen_data_series[str(_)] = generation_data[gen].copy()
        gen_names[str(_)] = gen

    # Create the optimisation model and set up constants/variables:
    R = range(len(contracted_energy))       # how many time intervals in total
    G = range(len(generation_data.columns))         # how many columns of generators

    m = Model()
    percent_of_generation = {}
    # Add a 'percentage' variable for each generator
    for g in G:
        percent_of_generation[str(g)] = m.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0)

    excess = [m.add_var(var_type=CONTINUOUS, lb=0.0) for r in R]
    unmatched = [m.add_var(var_type=CONTINUOUS, lb=0.0, ub = contracted_energy.max()) for r in R]
    hybrid_gen_sum = [m.add_var(var_type=CONTINUOUS, lb=0.0) for r in R]
    oversupply_flip_var = m.add_var(var_type=BINARY)

    # add the objective: to minimise firming (unmatched) - add price here?? (Yes)
    m.objective = minimize(xsum((unmatched[r] + unmatched[r]*wholesale_prices[r] + excess[r]*excess_penalty + oversupply_flip_var*market_cap) for r in R))

    # Add to hybrid_gen_sum variable by adding together each generation trace by the percentage variable
    for r in R:
        m += hybrid_gen_sum[r] <= sum([gen_data_series[str(g)][r] * percent_of_generation[str(g)] for g in G])
        m += hybrid_gen_sum[r] >= sum([gen_data_series[str(g)][r] * percent_of_generation[str(g)] for g in G])

    for r in R:
        m += unmatched[r] >= contracted_energy[r] - hybrid_gen_sum[r]
        m += excess[r] >= hybrid_gen_sum[r] - contracted_energy[r]

    # If total percent is constrained (i.e., PaP contract), add here:
    if constrain_total_percent:
        m += sum([percent_of_generation[str(g)] for g in G]) <= 1
        m += sum([percent_of_generation[str(g)] for g in G]) >= 1

    m += oversupply_flip_var >= xsum(hybrid_gen_sum[r] for r in R) / total_sum

    # TODO: think some more about inputs for shaped profiles etc here - this is 
    # getting confusing. 

    # Add constraint around CFE matching percent:
    if contract_type == '24/7':
        m += xsum(unmatched[r] for r in R) / total_sum <= (1 - cfe_score_min)

    m.optimize()

    hybrid_trace = pd.DataFrame(generation_data)
    hybrid_trace['Hybrid'] = 0
    check = True

    for g in G:
        try:
            hybrid_trace['Hybrid'] += gen_data_series[str(g)] * percent_of_generation[str(g)].x
        except:
            print('CFE score not reachable with this mix of generators.')
            check = False
    
    results = {}
    for g in G:
        name = gen_names[str(g)]
        try:
            details = {
                'Percent of generator output' : percent_of_generation[str(g)].x,
                'Percent of hybrid trace' : round(
                    sum(percent_of_generation[str(g)].x * gen_data_series[str(g)]) / sum(hybrid_trace['Hybrid']) * 100, 1)
            }
        except:
            print('CFE score not reachable with this mix of generators.')
            details = ''
            check = False

        results[name] = details

    # clear the model at end of run
    # Add some checks to make sure optimisation is running correctly
    
    if check:
        check_df = pd.DataFrame()
        check_df['Contracted'] = contracted_energy.copy()
        check_df['Hybrid Gen'] = hybrid_trace['Hybrid'].copy()
        check_df['Unmatched'] = [unmatched[r].x for r in R]
        check_df['Excess'] = [excess[r].x for r in R]

        check_df['Real Unmatched'] = (check_df['Contracted'] - check_df['Hybrid Gen']).clip(lower=0.0)
        check_df['Real Excess'] = (check_df['Hybrid Gen'] - check_df['Contracted']).clip(lower=0.0)

        check_df['Check unmatched'] = (check_df['Real Unmatched'].round(3) == check_df['Unmatched'].round(3))
        check_df['Check excess'] = (check_df['Real Excess'].round(3) == check_df['Excess'].round(3))

        check_df = check_df[(check_df['Check unmatched'] == False) | (check_df['Check excess'] == False)]

        assert check_df.empty == True, "Something is wrong with this optimisation - check constraints"

    m.clear()

    return hybrid_trace['Hybrid'], results

In [242]:
# Function to set up contract delivery structure and pass on to next stage (get
# optimal hybrid of generation traces). This function will take in the contract
# delivery type, and return the necessary input fields  to collect information
# about the contract definition.
def select_delivery_structure(
        contract_type:str
) -> pd.DataFrame:
    # TODO: fill this out
    valid_options = {'Pay as Produced', 'Pay as Consumed', 'Shaped', 'Baseload', '24/7'}
    if contract_type not in valid_options:
        raise ValueError(f'contract_type must be one of {valid_options}')
    
    return

# DOING - NOT FINISHED YET
def hybrid_shaped(
        redef_period:str,
        contracted_amount:float, 
        df:pd.DataFrame,
        generator_list:list[str],
        interval:str,
        percentile_profile:float
) -> pd.DataFrame:
    
    if contracted_amount < 0 or contracted_amount > 100:
        raise ValueError('contracted_amount must be a float between 0 - 100')
    
    if percentile_profile < 0 or percentile_profile > 1.0:
        raise ValueError('percentile_profile must be a float between 0 - 1.0.')
    
    df['Contracted Energy'] = df['Load'].copy()

    # use only the first year of data to create the contract basis. 
    # if there is only one year?
    num_intervals_in_day = int(24 / (interval / 60))  # hours in day / minutes / minutes in hour

    # also need to find out if it's a leap year:
    leap_year = check_leap_year(df, num_intervals_in_day)
    first_year = df.iloc[:num_intervals_in_day * (365 + leap_year)].copy()

    # Get the load and gen:
    first_year_load = first_year['Load']
    first_year_gen = first_year[generator_list].copy()

    # sum of total load in first year:
    first_year_load_sum = first_year_load.sum(numeric_only=True)

    # Create a new df to hold the shaped (percentile) profiles, make sure timestamps
    # all line up.
    gen_year_one_shaped = pd.DataFrame()
    gen_year_one_shaped['DateTime'] = pd.date_range(
        first_year_load.index[0], 
        first_year_load.index[-1], 
        freq='H'
    )

    # TODO: add commenting detail here to explain what's going on!!
    if redef_period == 'M':
        percentile_profile_period = first_year_gen.groupby(
            [first_year_gen.index.month.rename('Month'), 
             first_year_gen.index.hour.rename('Hour')]
        ).quantile(percentile_profile)

        gen_year_one_shaped['Month'] = gen_year_one_shaped.DateTime.dt.month
        gen_year_one_shaped['Hour'] = gen_year_one_shaped.DateTime.dt.hour

        gen_year_one_shaped = gen_year_one_shaped.set_index(['Month', 'Hour'])
        gen_year_one_shaped = pd.concat([gen_year_one_shaped , percentile_profile_period], axis='columns')
        gen_year_one_shaped = gen_year_one_shaped.reset_index().drop(columns=['Month', 'Hour'])

    
    if redef_period == 'Q':
        percentile_profile_period = first_year_gen.groupby(
            [first_year_gen.index.quarter.rename('Quarter'), 
             first_year_gen.index.hour.rename('Hour')]
        ).quantile(percentile_profile)

        gen_year_one_shaped['Quarter'] = gen_year_one_shaped.DateTime.dt.quarter
        gen_year_one_shaped['Hour'] = gen_year_one_shaped.DateTime.dt.hour

        gen_year_one_shaped = gen_year_one_shaped.set_index(['Quarter', 'Hour'])
        gen_year_one_shaped = pd.concat([gen_year_one_shaped , percentile_profile_period], axis='columns')
        gen_year_one_shaped = gen_year_one_shaped.reset_index().drop(columns=['Quarter', 'Hour'])


    if redef_period == 'Y':
        percentile_profile_period = first_year_gen.groupby(
            first_year_gen.index.hour.rename('Hour')
        ).quantile(percentile_profile)

        gen_year_one_shaped['Hour'] = gen_year_one_shaped.DateTime.dt.hour

        gen_year_one_shaped = gen_year_one_shaped.set_index('Hour')
        gen_year_one_shaped = pd.concat([gen_year_one_shaped , percentile_profile_period], axis='columns')
        gen_year_one_shaped = gen_year_one_shaped.reset_index().drop(columns=['Hour'])

    gen_year_one_shaped = gen_year_one_shaped.set_index('DateTime')

    hybrid_trace_series, percentages = run_hybrid_optimisation(
        contracted_energy=first_year['Contracted Energy'].copy(),
        wholesale_prices=first_year['RRP: QLD1'].copy(),
        generation_data=gen_year_one_shaped.copy(),
        excess_penalty=50,
        constrain_total_percent=False,
        total_sum=first_year_load_sum
    )

    print(percentages)

    return

# I THINK DONE
def hybrid_baseload(
        redef_period:str,
        contracted_amount:float, 
        df:pd.DataFrame,
        generator_list:list[str],
        interval:str,
        percentile_profile:float
) -> pd.DataFrame:

    if contracted_amount < 0:
        raise ValueError('contracted_amount must be greater than 0.')
    
    # use only the first year of data to create the contract basis. 
    # if there is only one year?
    num_intervals_in_day = int(24 / (interval / 60))  # hours in day / minutes / minutes in hour

    # also need to find out if it's a leap year:
    leap_year = check_leap_year(df, num_intervals_in_day)
    first_year = df.iloc[:num_intervals_in_day * (365 + leap_year)].copy()

    # Resample to hourly load, then take the hourly average per chosen period
    first_year_load = first_year['Load'].copy()

    # Use a map to allocate hourly values across all years of load data:
    if redef_period == 'Y':
        avg_hourly_load = first_year_load.mean(numeric_only=True)

        # the contracted energy needs to be updated by the contracted_amount percentage:
        df['Contracted Energy'] = round(avg_hourly_load) * (contracted_amount / 100)
    
    else:
        # the contracted energy needs to be updated by the contracted_amount percentage:
        avg_hourly_load = pd.DataFrame(first_year_load.resample(redef_period).mean(numeric_only=True) * (contracted_amount / 100))
        avg_hourly_load['Load'] = avg_hourly_load['Load'].round()
        avg_hourly_load['M'] = avg_hourly_load.index.month
        avg_hourly_load['Q'] = avg_hourly_load.index.quarter

        map_dict = dict(zip(avg_hourly_load[redef_period], avg_hourly_load['Load']))

        first_year['M'] = first_year.index.month
        first_year['Q'] = first_year.index.quarter

        first_year['Contracted Energy'] = first_year[redef_period].copy()
        first_year['Contracted Energy'] = first_year['Contracted Energy'].map(map_dict)

        first_year = first_year.drop(columns=['M', 'Q'])
    
    hybrid_trace_series, percentages = run_hybrid_optimisation(
        contracted_energy=first_year['Contracted Energy'].copy(),
        wholesale_prices=first_year['RRP: QLD1'].copy(),
        generation_data=first_year[generator_list].copy(),
        excess_penalty=0.0,
        constrain_total_percent=False,
        total_sum=first_year_load.sum(numeric_only=True)
    )

    first_year = pd.concat([first_year, hybrid_trace_series], axis='columns')
    return first_year

# STARTED
def hybrid_247(
        redef_period:str,
        contracted_amount:float, 
        df:pd.DataFrame,
        generator_list:list[str],
        interval:str,
        percentile_profile:float
) -> pd.DataFrame:

    if contracted_amount < 0 or contracted_amount > 100:
        raise ValueError('contracted_amount must be a float between 0-100')

    # use only the first year of data to create the contract basis. 
    # if there is only one year?
    num_intervals_in_day = int(24 / (interval / 60))  # hours in day / minutes / minutes in hour

    # also need to find out if it's a leap year:
    leap_year = check_leap_year(df, num_intervals_in_day)
    first_year = df.iloc[:num_intervals_in_day * (365 + leap_year)].copy()

    # Get first year load (and total sum):
    first_year_load = first_year['Load'].copy()
    first_year_load_sum = first_year_load.sum(numeric_only=True)

    hybrid_trace_series, percentages = run_hybrid_optimisation(
        contracted_energy=first_year['Load'].copy(),
        wholesale_prices=first_year['RRP: QLD1'].copy(),
        generation_data=first_year[generator_list].copy(),
        excess_penalty=0.0,
        constrain_total_percent=False,
        total_sum=first_year_load.sum(numeric_only=True),
        contract_type='24/7',
        cfe_score_min=contracted_amount/100
    )
    
    print(percentages)

    return


def hybrid_pap(
        redef_period:str, 
        contracted_amount:float, 
        df:pd.DataFrame,
        interval:str
) -> pd.DataFrame:
    
    # TODO: validate contracted amount here!

    return


def hybrid_pac(
        redef_period:str,
        contracted_amount:float, 
        df:pd.DataFrame,
        interval:str
) -> pd.DataFrame:
    
    # TODO: validate contracted amount here!

    return



def create_hybrid_generation(
        contract_type:str, # describes contract delivery structure
        redef_period:str, # one of python's offset strings indicating when the contract gets "redefined"
        contracted_amount:float, # a number 0-100(+) indicating a percentage. Definition depends on contract type.
        df:pd.DataFrame, # df containing Load, all gen profiles, prices, emissions.
        generator_list:list[str],
        interval:str, # time interval in minutes that data is currently in
        percentile_profile:float=0 # for Shaped contracts only: to define the percentile of generation profiles to match.
) -> pd.DataFrame:
    
    valid_contracts = {'Pay as Produced', 'Pay as Consumed', 'Shaped', 'Baseload', '24/7'}
    if contract_type not in valid_contracts:
        raise ValueError(f'contract_type must be one of {valid_contracts}')
    
    valid_periods = {'M', 'Q', 'Y'}
    if redef_period not in valid_periods:
        raise ValueError(f'redef_period must be one of {valid_periods}')

    opt_hybrid_funcs = {
        'Pay as Produced' : hybrid_pap, 
        'Pay as Consumed' : hybrid_pac,
        'Shaped' : hybrid_shaped, 
        'Baseload' : hybrid_baseload, 
        '24/7' : hybrid_247
    }

    df_with_hybrid = opt_hybrid_funcs[contract_type](redef_period, contracted_amount, df, generator_list, interval, percentile_profile)

    return df_with_hybrid


In [245]:
# TODO: update df to only use useful columns - > this will be filtered by scenario!!

hybrid_profiles_test = create_hybrid_generation('24/7', 'M',  50, combined_data_firming, ['MEWF1: Wind - Onshore', 'SMCSF1: Photovoltaic Flat panel'], interval)
# hybrid_profiles_test.head()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-06
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Set parameter NodeLimit to value 1073741824
Set parameter SolutionLimit to value 1073741824
Set parameter IntFeasTol to value 1e-06
Set parameter Method to value 3
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 69698 rows, 52275 columns and 184965 nonzeros
Model fingerprint: 0x1ed59383
Variable types: 52274 continuous, 1 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e-07, 4e+02]
  Objective range  [1e+00, 3e+08]
  Bounds range     [1e+00, 5e+03]
  RHS range        [5e-01, 5e+03]
Presolve removed 19003 rows and 17424 columns
Presolve time: 0.01s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.02 work units)
Thread count was 1 (of 8 available processors)

Solu